In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager
from graphrag.query.indexer_adapters import (
    read_indexer_communities,
    read_indexer_entities,
    read_indexer_reports,
)
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

In [2]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [3]:
api_key = os.environ["AZURE_OPENAI_API_KEY"]
llm_model = "gpt-4.1-mini" #os.environ["GRAPHRAG_LLM_MODEL"]

config = LanguageModelConfig(
    api_key=api_key,
    type=ModelType.AzureOpenAIChat,
    api_base=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_version="2025-04-01-preview",
    deployment_name=llm_model,
    model=llm_model,
    max_retries=20,
)
model = ModelManager().get_or_create_chat_model(
    name="global_search",
    model_type=ModelType.AzureOpenAIChat,
    config=config,
)

token_encoder = tiktoken.encoding_for_model(llm_model)

In [4]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../research/output"
COMMUNITY_TABLE = "communities"
COMMUNITY_REPORT_TABLE = "community_reports"
ENTITY_TABLE = "entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [5]:
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")

communities = read_indexer_communities(community_df, report_df)
reports = read_indexer_reports(report_df, community_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, community_df, COMMUNITY_LEVEL)

print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)

report_df.head()

Total report count: 997
Report count after filtering by community level 2: 880


,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,219aa0adad0a4c2a8c257a9a8fc9a25b,995,995,6,993,[],Societe Generale Group Financial and Risk Mana...,This community centers on the Societe Generale...,# Societe Generale Group Financial and Risk Ma...,8.2,The high impact severity rating reflects the G...,[{'explanation': 'The Societe Generale Group (...,"{\n ""title"": ""Societe Generale Group Financ...",2025-09-20,98
1,2b5db159c9df468e9d98c24171df8529,996,996,6,993,[],Global Markets Business Units and Global Banki...,This community consists of two key business un...,# Global Markets Business Units and Global Ban...,6.5,The impact severity rating is moderately high ...,[{'explanation': 'The Global Markets Business ...,"{\n ""title"": ""Global Markets Business Units...",2025-09-20,2
2,ba30bb594eae4ffa9ac7148a972acb3e,993,993,5,989,"[995, 996]",Societe Generale Group Financial and Risk Mana...,This community centers on the Societe Generale...,# Societe Generale Group Financial and Risk Ma...,8.5,The high impact severity rating reflects the G...,[{'explanation': 'The Societe Generale Group i...,"{\n ""title"": ""Societe Generale Group Financ...",2025-09-20,100
3,9fba254b36864ffead70f03c116383ad,994,994,5,989,[],Derecognition and Pass-Through Agreement,This community centers on the financial accoun...,# Derecognition and Pass-Through Agreement\n\n...,4.2,The impact severity rating is moderate due to ...,[{'explanation': 'Derecognition is a fundament...,"{\n ""title"": ""Derecognition and Pass-Throug...",2025-09-20,2
4,a7eb55d0b848464d8efe29e8cd8acf86,985,985,4,880,[],Germany's Financial and Real Estate Ecosystem ...,This community centers on Germany as a pivotal...,# Germany's Financial and Real Estate Ecosyste...,8.2,The impact severity rating is high due to Germ...,[{'explanation': 'Germany is a key member of t...,"{\n ""title"": ""Germany's Financial and Real ...",2025-09-20,12


In [6]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    communities=communities,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [7]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [8]:
search_engine = GlobalSearch(
    model=model,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [9]:
prompt = """프랑스 은행 BNP 에 대해서 아래의 보고서 형태로 작성해 주시오.

# 보고서 항목
주요 리스크 요인
- 산업 구조적 리스크 
- 규제 변화로 인한 수익성 악화 가능성
- 기업별 특수 리스크

# 언어 및 스타일
- 한국어로 간결하고 명확하게 작성하십시오
- 최대한 객관적이되 통찰력 있는 시각으로 작성하십시오.
- 대한민국 경제 뉴스 브리핑이나 토론에서 사용하는 전문적인 단어를 사용하십시오.
"""

In [10]:
result = await search_engine.search(prompt)

print(result.response)

# BNP 파리바 주요 리스크 요인 보고서

## 1. 산업 구조적 리스크

BNP 파리바는 글로벌 금융시장 내 다각화된 사업 포트폴리오를 보유하고 있으며, 유럽, 아시아, 중동, 미주 등 광범위한 지역에서 은행, 리스, 보험, 자산관리 등 다양한 금융 서비스를 제공하고 있다. 이러한 복합적 산업 구조는 시장 변동성, 지정학적 불확실성, 그리고 각국 경제 상황 변화에 민감하게 반응할 수밖에 없는 구조적 리스크를 내포한다.

특히 중국 시장에서는 자동차 산업 부진과 비경상적 손실이 지속되며 금융 자산 가치 하락과 수익성 저하 우려가 존재한다. 영국 금융 및 연금 시장에서는 복잡한 규제 환경과 연금 수급 구조 변화가 장기적 불확실성을 야기하고 있다. 중부 유럽의 핵심 금융 허브인 폴란드에서는 스위스 프랑 모기지 대출 관련 대규모 법적 분쟁과 환율 변동성이 금융 안정성에 부담으로 작용하고 있다.

또한, 우크라이나 전쟁과 같은 지정학적 리스크는 러시아 및 우크라이나 지역 자회사에 대한 신용 리스크를 증대시키며, 글로벌 경제 환경 변화에 따른 금융시장 변동성 확대도 BNP 파리바의 사업 안정성에 부정적 영향을 미칠 수 있다. 이처럼 다국적 금융그룹으로서의 복잡한 산업 구조는 내부 통제 및 리스크 관리의 어려움을 가중시키는 요인으로 작용한다 [Data: Reports (251, 398, 302, 424, 57, +more)].

## 2. 규제 변화로 인한 수익성 악화 가능성

BNP 파리바는 유럽연합(EU) 및 각국 금융당국의 강화된 자본규제, 리스크 관리 기준, 소비자 보호법, 그리고 국제회계기준(IFRS) 변화에 직면해 있다. 특히 바젤 III 및 IV, CRR/CRD IV, CRR3/CRD6 등 엄격한 자본 및 유동성 규제는 자본비용 상승과 대출 여력 축소를 초래하며, 이는 은행의 영업 마진과 수익성에 직접적인 압박 요인으로 작용할 가능성이 크다.

2024년부터 프랑스 금융안정위원회(HCSF)의 반사이클 자본 완충금 상향(0.5%→1%)과 벨기에, 이탈리아 등

In [11]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,398,United Kingdom Financial and Pension Managemen...,0.063158,# United Kingdom Financial and Pension Managem...,7.5
1,251,BNPP-Linked Financial Entities and Institution...,0.042105,# BNPP-Linked Financial Entities and Instituti...,7.5
2,385,BNP Paribas Group Economic Research and Macroe...,0.007895,# BNP Paribas Group Economic Research and Macr...,8.5
3,155,Societe Generale Finance and Human Resources G...,0.007895,# Societe Generale Finance and Human Resources...,6.5
4,190,SCI Etampes Notre-Dame and French Retail & Pri...,0.005263,# SCI Etampes Notre-Dame and French Retail & P...,4.5
...,...,...,...,...,...
875,700,Andalan Multi Guna PT and BNPP Entities in Ind...,0.010526,# Andalan Multi Guna PT and BNPP Entities in I...,6.5
876,406,BNP Paribas and AXA IM Project Community,0.005263,# BNP Paribas and AXA IM Project Community\n\n...,7.5
877,106,S&P Global and Financial Risk Metrics Community,0.002632,# S&P Global and Financial Risk Metrics Commun...,8.2
878,333,Societe Generale Group Entities and Hyperinfla...,0.015789,# Societe Generale Group Entities and Hyperinf...,7.5
